# Introduction au langage Prolog

Prolog, pour programmation en logique est un langage créé par A. Colmerauer, en 1972 à Marseille. La programmation logique est un paradigme différent de la programmation impérative ou de la programmation fonctionnelle. On définit des faits élémentaires et des règles de la manière suivante :

```
apprend(eve, mathematiques).
apprend(benjamin, informatique).
apprend(benjamin, physique).
enseigne(alice, physique).
enseigne(pierre, mathematiques).
enseigne(pierre, informatique).

etudiant_de(E,P):-apprend(E,M), enseigne(P,M).
```

Ces informations décrivent l'ensemble des connaissances du programme. On lit ici "Benjamin apprend l'informatique", ou "Alice enseigne la physique". La dernière ligne est une règle : "L'élève E est étudiant du professeur P si E apprend la matière M et P enseigne M". Le symbole `:-` se lit *si*, et la virgule entre `apprend(E,M)` et `enseigne(P,M)` sinifie *et*. Ensuite l'utilisateur peut effectuer des requêtes comme ceci : 

`?-etudiant_de(E, pierre)`

L'interpréteur Prolog renvoie alors `E = benjamin` et `E = eve`.

Ce qui différencie la programmation logique des formes plus courantes d'informatique impérative, c'est qu'on n'explique pas au programme comment résoudre la requête, on se contente de décrire le problème.

De très nombreuses introductions au langage Prolog peuvent être trouvées sur internet, je n'en cite qu'une seule qui m'est apparue comme convenable pour une première approche : <cite data-cite="Flach">(Flach)</cite>.

Un autre ouvrage plutôt plus complet est <cite data-cite="Sterling">(Sterling)</cite>.

Dans la suite j'essaie d'implémenter moi même un interpréteur d'un sous-ensemble du langage Prolog, en utilisant le langage OCaml qui a la particularité d'être principalement un langage fonctionnel.

# La grammaire du langage Prolog

La première chose à faire est d'étudier la grammaire du langage, c'est à dire comment sont formés les mots du langage (en linguistique on parle de morphologie), mais aussi comment peuvent s'agencer les mots pour former un programme valide (c'est la syntaxe). 

La grammaire du langage Prolog est non-contextuelle, ce qui signifie que toutes les règles ne contiennent à gauche qu'un seul symbole. Plus intuitivement, pour décrire un symbole non-terminal (entre chevrons) on a pas besoin de connaitre ce qu'il y a avant ou après, d'où le "non-contextuel". J'en suis arrivé à cette définition de la grammaire de mon sous-ensemble du langage Prolog, ici présentée sous la forme de Backus-Naur :

```
<Caractère>    ::=  'a'..'z' | 'A'..'Z' | '_' | '0'..'9'
<Mot>          ::=  <Caractère> | <Caractère> <Mot>
<Prédicat>     ::=  'a'..'z' | 'a'..'z' <Mot>
<Variable>     ::=  'A'..'Z' | 'A'..'Z' <Mot>
<Programme>    ::=  <Clause> | <Clause> <Programme>
<Clause>       ::=  <Terme> '.' | <Terme> ':-' <ListeTermes> '.'
<ListeTermes>  ::=  <Terme> | <Terme> ',' <ListeTermes>
<Terme>        ::=  <Variable> | <Predicat> | <Predicat> '(' <ListeTermes> ')' | <Tableau>
<Tableau>      ::=  '[]' | '[' <ListeTermes> ']' 
            | '[' <ListeTermes> '|' <Tableau> ']' | '[' <ListeTermes> '|' <Variable> ']'
```

Les grammaires non-contextuelles sont équivalentes aux automates à pile : ces derniers sont plus puissants que les automates finis (équivalents aux expressions régulières), mais moins puissants que les machines de Turing. 

Une sous classe des grammaires non contextuelles est celle des grammaires non-contextuelles déterministes. Une grammaire non contextuelle est dit déterministe si elle est équivalente à un automate à pile déterministe. Cette sous catégorie est intéressante puisqu'il existe alors des algorithmes en temps linéaire pour effectuer l'analyse syntaxique... Je suppose que cette grammaire de Prolog est bien déterministe, mais je ne sais pas le montrer.

De toute façon la structure de l'analyseur syntaxique que je vais utiliser me permet en théorie d'utiliser n'importe quelle grammaire non-contextuelle.

# La représentation des programmes Prolog

C'est probablement la partie la plus importante du design de l'interpréteur Prolog, puisque la manière de stocker les programmes Prolog en Caml impactera toute la suite.

```ocaml
type var = Id of string * int
type term = Var of var | Predicate of string * (term list)
type clause = Clause of term * (term list);;
```

Cette première structure permet de représenter le plus petit Prolog possible. Chaque variable est représentée par une chaîne de caractères, son nom, et un numéro. Le numéro permet un renommage facile des variables, qui est nécessaire pour éviter des conflits noms dans l'algorithme d'unification. Le type term est le plus important, un terme peut être une variable, où une structure composée d'une chaîne, le prédicat et une liste de termes. Ainsi `apprend(eve, mathématiques)` est un terme. Finalement une clause est un terme, qui est à gauche du symbole `:-`, et la liste des termes qui sont à droite. Un programme sera alors une liste de clauses, séparées par des points.

Ce n'est pas moi qui ai inventé cette structure particulière : je l'ai retrouvée dans un diapo de cours en Haskell sur le langage Prolog, <cite data-cite="Smaill">(Smaill)</cite>. Une structure proche est utilisée en Lisp par Peter Norvig dans <cite data-cite="Norvig">(Norvig)</cite>.

```ocaml
type var = Id of string * int
type table = Empty | NonEmpty of term * table | TVar of var
and term = Var of var | Predicate of string * (term list) | Table of table
type clause = Clause of term * (term list);;
```

L'ajout des listes Prolog (je les ai appelées tables ici pour les différencier des listes du langage Caml, mais ce n'est probablement pas le meilleur nom), rend la structure sensiblement plus complexe. J'ai choisi ce design pour pouvoir modéliser facilement une table comme `[a, b, c | X]`. Ici `X` est une variable qui ne peut que représenter une table, on ne veut pas que cela puisse être un terme quelconque (comme `eve` exemple). C'est une très bonne chose de rendre impossible la représentation de programmes invalides dans le type des objets, mais cela rend certaines choses plus compliquées. Les variables sont présentes à différents endroits, et certaines ne peuvent représenter que des tableaux alors que d'autres peuvent représenter n'importe quel terme.

J'ai mis un moment (plus ou moins les deux derniers mois) à me décider sur cette structure, et c'est parce que j'ai dû la changer un certain nombre de fois que j'ai réécrit beaucoup de choses pour qu'elles soient un peu plus modulables.

# L'analyse syntaxique

Si j'ai bien appris une chose en écrivant un premier parser moi même sans reprendre de grandes idées existantes, c'est qu'il mieux vaut bien réfléchir avant si on veux être capable de modifier, ou même de relire son code après. Maintenant j'ai compris que le parsing est un art en soi, et j'ai décidé d'écrire un analyseur récursif descendant utilisant des combinateurs. J'ai appris les idées dans <cite data-cite="Ljunglof">(Ljunglöf)</cite>.

Je vois ici un parser comme une fonction qui à une chaîne de caractères à analyser renvoie un couple composé de la chaîne restante à analyser et une structure, un arbre produit par l'analyseur qui représente les caractères analysés. L'idée des combinateurs est de créer de nouveaux parsers plus complexes à partir de ceux existants. Voici la structure que produit le parser :

```ocaml
type structure = S of string | V of var | L of table | T of term | C of clause 
                | W of clause list | P of string | TL of term list
```

Les parsers élémentaires que j'ai écrit directement permettent de reconnaître un caractère passé en paramètre et renvoient une structure `S chaîne` avec chaîne étant le caractère reconnu. J'en ai écrit d'autres permettant de reconnaître n'importe quelle lettre majuscule, ou minuscule, ou encore n'importe quel caractère valide.

Ensuite j'utilise deux combinateurs : `<+>` et `<*>`. Pour des raisons de priorité des opérateurs en OCaml, je les ai renommés `+~+` et `*~*` respectivement. Le combinateur `<+>` exprime le *ou* `|` des règles de grammaire. Le combinateur `<*>` quant à lui premet de chaîner les parsers, il n'est pas représenté dans les règles de grammaire, les symboles sont juste accolés. Ainsi la règle `<Mot> ::= <Caractère> | <Caractère> <Mot>` se traduirait par : 
```ocaml
let rec mot = caractère +~+ caractère *~* mot
```
Malheureusement, on ne peut pas exprimer directement la récursion ainsi... Même si cela fonctionne très bien pour les règles non récursives, une limitation de OCaml —pour éviter de déclarer des structures comme `let rec a = a`— impose d'écrire quelques lignes en plus, rendant le code moins lisible.

Le dernier opérateur que je définis est `>~>`, d'une manière très similaire au *bind* `>>=` monadique. Cet opérateur prend à gauche un parser, et à droite une fonction d'une liste de structures vers une structure (en simplifiant). La fonction peut par exemple décrire comment construire un nouveau mot à partir d'un caractère et d'un mot.

# Les fonctions de base sur les programmes

* `string_of_term : term -> string` Transforme un terme en une chaîne de caractères.

* `var_in_term : var -> term -> bool` Teste si une variable est dans un terme.

* `var_in_eql : var -> (term * term) list -> bool` Teste si une variable est dans une liste de couples de termes.

* `replace_var_in_term : var -> term -> term -> term` Remplace une variable par un terme dans un terme. Attention, la fonction échoue si on essaie de remplacer une variable qui est censée représenter un tableau par un terme qui n'est pas un tableau.

* `replace_var_in_eql : var -> term -> (term * term) list -> (term * term) list` Remplace une variable par un terme dans une liste de couples de termes. Mêmes limitations.

* `find_vars_in_termlist : term list -> term list` Liste toutes les variables d'une liste de termes.

* `find_tvars_in_term : term -> var list` Liste toutes les variables représentant un tableau dans un terme.

* `find_tvars_in_termlist : term list -> var list` Liste toutes les variables représentant un tableau dans une liste de termes.

* `find_tvars_in_clause : clause -> var list` Liste toutes les variables représentant un tableau dans une clause.


# La correction des types

Le problème est qu'une même variable ne peut pas représenter n'importe quel terme si l'on sait que la variable ne peut représenter que des tableaux. La solution est de rechercher toutes les variables qui ne peuvent représenter que des tableaux, et forcer toutes leurs occurences par des variables qui ne peuvent représenter que des tableaux.

Ensuite j'ai juste écrit des fonctions composant le parser et le vérificateur de types.

Dans toute la suite du code il ne faut jamais qu'une variable appraisse à un endroit comme pouvant représenter n'importe quel terme et dans un autre comme pouvant ne représenter que des tableaux.

# L'algorithme derrière Prolog

Ma référence en ce qui concerne l'algorithme utilisé par Prolog est <cite data-cite="NilssonMaluszynski">(NilssonMaluszynski)</cite>. 

Ce qui semble être le premier article traitant du type de résolution utilisé par Prolog est <cite data-cite="Robinson">(Robinson)</cite>. Cet article introduit l'unification, et le *principe de résolution* (Resolution Principle) dont dérive la SLD-resolution (Linear resolution for Definite clauses with Selection function) utilisée par Prolog.

Une *clause définie*, aussi nommée clause de Horn est de la forme `(P1 et P2 et ... et Pn) => Q`. En particulier, `(non P) => Q` n'est pas une clause de Horn. On peut par contre exprimer`(P1 ou P2) => Q` avec des clauses de Horn, il suffit d'écrire les deux clauses `P1 => Q` et `P2 => Q`. Les versions premières de Prolog se limitent aux clauses de Horn, pour une principale raison : la *correction* et la *complétude* (*soundness* and *completeness*) ont été montrée pour la SLD-resolution, qui n'est valide que sur des clauses de Horn. Pour les citations (un peu compliquées) : la SLD-resolution est introduite par Kowalski en 1974, la correction est montrée par Clark en 1979, la complétude a été montrée premièrement par Hill en 1974, mais quelque chose de plus fort a été montré par Clark en 1979. 


# L'unification

L'unification se fait entre deux termes A et B. Deux termes s'unifient s'il existe une substitution `thêta` des variables de A telle que `B = thêta(A)`. 

Soit la clause du programme Prolog `etudiant_de(E,P):-apprend(E,M), enseigne(P,M).`. Si l'on cherche à réaliser la requête `?-étudiant_de(E, pierre)`, on unifie `étudiant_de(E, pierre)` avec `etudiant_de(E,P)`, le membre de gauche de la clause. La substitution thêta remplace `P` par `pierre` et ne modifie pas les autres variables. Pour continuer la recherche, on applique la substitution au termes de droite de la clause.

Voici l'algorithme décrit dans <cite data-cite="NilssonMaluszynski">(NilssonMaluszynski)</cite> :
```
E est l'ensemble des équations. Au départ il n'y en a qu'une seule : 
Par exemple E = {etudiant_de(E,P) = étudiant_de(E, pierre)}.

Répéter tant que E change 
(jusqu'à ce que l'on ne puisse plus rien appliquer aux équations)
    Sélectionner une équation  s = t dans E;
    Si s = t est de la forme :
        f(s1, ..., sn) = f(t1, ..., tn) avec n >= 0 
            Alors remplacer l'équation par s1=t1 ... sn=tn
        f(s1, ..., sm) = g(t1, ..., tn) avec f != g 
            Alors ÉCHEC
        X = X 
            Alors supprimer l'équation
        t = X où t n'est pas une variable 
            Alors remplacer l'équation par X = t
        X = t où X != t et X apparait plus d'une fois dans E 
            Si X est un sous-terme de t Alors ÉCHEC
            Sinon on remplace toutes les autres occurences de X par t
```

Il est prouvé que cet algorithme termine et renvoie soit échec, soit un ensemble équivalent des équations sous *forme résolue*. Un ensemble d'équations `{X1 = t1, ..., Xn = tn}` est dit sous forme résolue si les `(Xn)` sont des variables, les `(tn)` sont des termes et aucune des variables `Xn` n'apparait dans les `tn`. Cela permet ensuite de déterminer un unifieur.

Remarques : 

* C'est dommage que l'algorithme se prête si bien à la programmation impérative quand on programme dans un langage fonctionnel. 

* On trouve des algorithmes pour déterminer un unifieur qui ne manipulent pas exactement comme ça des systèmes d'équations, (et qui sont dans des styles plus fonctionnels), mais cet algorithme est le seul que j'arrive à comprendre convenablement, et en plus <cite data-cite="NilssonMaluszynski">(NilssonMaluszynski)</cite> prouve sa terminaison et sa correction, ce qui est très bon point.

* Le test `Si X est un sous-terme de t Alors ÉCHEC` est en pratique pas réalisé dans la plupart des implémentations Prolog (pas comme ça en tout cas), il est très lent et le cas n'arrive que très peu en pratique. On peut donc avoir des boucles infinies... Les versions "modernes" gèrent les unifications de structures infinies... Une citation de <cite data-cite="Norvig">(Norvig)</cite> "*This represents a circular, infinite unification. Some versions of Prolog, notably Prolog II (Giannesini et al. 1986), provide an interpretation for such structures, but it is tricky to define the semantics of infinite structures.*"

À la place de manipuler des ensembles (`Set` existe en OCaml), j'ai juste utilisé des liste triées. La fonction `compare` est magique, elle définit une relation d'ordre pour n'importe quel type. J'ai ici utilisé la version complète de l'algorithme, qui n'est certainement pas la plus efficace. Comme l'a dit Donald Knuth : "*Premature optimization is the root of all evil (or at least most of it) in programming.*"

On définit une substitution comme une liste de couples (var,term), le terme est inséré à la place de la variable.

Si l'on sait que `etudiant_de(E1,P):-apprend(E1,M), enseigne(P,M).` (c) et que l'on veut montrer `étudiant_de(E0, pierre).` (r), alors il faut trouver une substitution qui unifie la tête de la clause (c) avec la requête (r).

Pour l'exemple plus haut, il faut la substitution `thêta = { P/pierre, E1/E0}`. Il faut comprendre "La variable P est remplacée par 'pierre', la variable E1 est remplacée par la variable E0. C'est bien un unifieur, si on l'applique sur la requête et sur la tête de la clause, les deux deviennent égales.

Il peut exister plusieurs unifieurs, par exemple `thêta2 = { P/pierre, E0/samuel, E1/samuel}` en est aussi un. On voit que thêta2, c'est thêta composé avec `{E0/samuel}`, on dit alors que thêta est plus général que thêta2. Nous, nous cherchons le plus général de ces unifieurs, le MGU pour *Most General Unifier*.

Un détail : on peut avoir thêta plus général que oméga, et oméga plus général que théta, la relation n'est donc pas antisymétrique. En fait, le MGU est unique au renommage des variables près.

On peut déduire facilement un MGU du système d'équations sous forme résolue de la fonction `solve` (c'est à ça qu'elle sert). Si `{X1 = t1, ..., Xn = tn}` est sous forme résolue, alors `{X1/t1, ..., Xn/tn}` est un MGU.

Lorsque l'on veut unifier `etudiant_de(E,P)` et `etudiant_de(E,pierre)`, les deux variables `E` ne doivent pas être nommées de la même manière.

Quelques explications sur les derniers deux exemples :

* Je sais que pour tout `Z`, `f(g(Z),Z)` est vraie, et j'aimerais savoir s'il existe des couples `(X,Y)` tels que `f(X,g(Y))` soit vraie. Le programme me répond oui, il suffit de choisir `X = g(g(Y))`

* Je sais que pour tout `(X,Y)`, `f(X,g(Y))` est vraie, et j'aimerais savoir s'il existe des `Z` tels que `f(g(Z),Z)` soit vraie. Le programme me répond oui, il suffit de choisir `Z = g(Y)`

# Le backtracking

Il me semble que nous nous rapprochons du but ! L'unification est une partie très importante de la SLD-resolution. L'autre point important est le backtracking. Encore quelques fonctions pour appliquer des substitutions sur des termes, listes de termes, pour composer des substitutions, pour vérifier que un terme et une clause n'ont pas de variables en commun, pour effectuer les renommages si nécessaire, et après je pense qu'il sera possible d'implémenter le backtracking.

Voilà la manière dont j'ai compris l'algorithme :

* On cherche à satisfaire une requête `<- A1,...An`.

* On a `A1` qui unifie avec `Hi`, où `Hi <- Ci_1,..Ci_m` est une la i-ème clause du programme. On a auparavant renommé toutes les variables de `Hi <- Ci_1,..Ci_m` qui étaient présentes dans `A1`, sinon on ne peut pas appeler `mgu`. On a donc un MGU `thêta1`.

* La nouvelle requête à satisfaire est `<- thêta1(Ci_1,...Ci_m, A2,...An)`.

* On récure, si on a à montrer `thêta_k( ... thêta2( thêta1( _vide_) ) ...)` alors c'est gagné (`_vide_` signifie toujours vrai), la composée `thêta_tot` des `thêta_k` est une substitution des variables de la requête de départ. Ce qu'on veut renvoyer à l'utilisateur c'est l'image des variables de `A1,...An` par `thêta_tot`.

* Si `A1` ne s'unifie avec aucun `Hi`, ça ne sert à rien d'essayer d'unifier `A2`, puisque de toute façon on garde `A1` dans la nouvelle requête. Alors il faut remonter. C'est à dire qu'il faut essayer les autres unifications à l'étape d'avant.

Comment renommer les clauses pour avoir des noms libres à chaque unification ? On peut utiliser le niveau de récursion comme identifiant, que l'on place dans l'entier transporté avec la variable.

* On veut des variables numérotées à 0 dans la requête. 

* On passe 1 lors du premier appel de sld : la première clause utilisée est renommée à 1, les variables de la requête sont toutes à 0.

* La deuxième clause utilisée est renommée à 2, dans la requête il y a des variables à 1 et à 0 : pas de conflit ! etc...

# Le Prolog aujourd'hui

J'ai beaucoup aimé la lecture de la préface de <cite data-cite="Sterling">(Sterling)</cite>. Robert Kowalski raconte son premier contact avec Prolog, le langage ayant été initié vers 1972 par Alain Colmerauer et Philippe Roussel à Marseille. Kowalski y est invité, mais les serveurs de calcul ne sont pas là, ils se connectent par telnet sur une machine IBM à Grenoble ! Pour la petite histoire, le premier message que Kowalski reçoit à l'exécution de son programme est "DEBORDEMENT DE PILE" !

Kowalski a introduit le SLD-résolution, entre autre nombreux travaux sur les algorithmes de Prolog. Il a eu deux étudiants en thèse. Keith Clark, qui a montré la correction et la complétude de la SLD-résolution et qui a introduit le principe de *négation as failure*. David H. D. Warren lui a écrit le premier compilateur pour Prolog, utilisant les *machines abstraites de Warren*.

# Que faire maintenant ?

* Beaucoup de tests pour vérifier si tout fonctionne.

* Peut-être nettoyer un peu le code...

* Écrire quelques fonctions comme une ligne de commande interactive, pour une utilisation plus facile. Rendre les résultats plus clairs.

* Proposer une deuxième version de la recherche, qui renvoie tous les résultats possibles, ou qui propose de chercher le résultat suivant.

* Je pensais essayer d'écrire un programme pour "*résoudre*" le Cluedo. J'avais déjà essayé sans succès en C++. Le problème semble plus adapté au langage Prolog : "*tel joueur possède telle carte*", "*si un joueur montre une carte à un autre pour réfuter une supposition (triplet de cartes), c'est qu'il possède une des trois cartes*", sont des règles que l'on peut écrire en Prolog. Malheureusement j'ai trouvé un super mémoire de Master <cite data-cite="Aartun">(Aartun)</cite> qui traite bien le sujet.

* Je regarde un peu comment est-ce que l'on peut sortir des clauses de Horn, mais c'est un sujet compliqué.

* Je voulais voir s'il est possible d'étendre à des langages fonctionels des éléments de programmation logique. C'est aussi un thème intéressant, qui se nomme *functional logic programming*. On trouve plusieurs petits langages sur internet qui utilisent des approches différentes, mais aucun d'eux ne semble bien abouti. Ce serait presque un idéal d'associer des mécanismes efficaces des langages fonctionnels comme l'évaluation paresseuse, et les capacités de la programmation logique pour décrire les problèmes...

* Pour compiler Prolog, il faut se tourner vers les *Machines abstraites de Warren*.